#Imports

In [1]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -U langchain-community
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

! pip install unstructured
! pip install rapidocr-onnxruntime

clear_output()

CPU times: user 1.28 s, sys: 208 ms, total: 1.49 s
Wall time: 3min 6s


In [2]:

%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time
from tqdm import tqdm

import langchain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, UnstructuredMarkdownLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import FAISS

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain.chains import RetrievalQA
from langchain_community.retrievers import TFIDFRetriever
from langchain.retrievers import EnsembleRetriever

import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)



CPU times: user 9.14 s, sys: 1.2 s, total: 10.3 s
Wall time: 18.6 s


# Chunks

In [7]:
!unzip -o full_doc.zip
clear_output()

In [8]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [9]:
loader = DirectoryLoader(
    'full_doc/',
    glob="./*.md",
    loader_cls=TextLoader,
    show_progress=True,
    use_multithreading=True,
)
documents = loader.load()


# loader = DirectoryLoader(
#     'full_doc/',
#     glob="./*.md",
#     loader_cls=UnstructuredMarkdownLoader,
#     show_progress=True,
#     use_multithreading=True,
# )


# documents = loader.load()

# documents = []
# for file in tqdm(os.listdir('full_doc/')):
#     path = 'full_doc/' + file
#     if file[-2:] == 'md':
#         loader = UnstructuredMarkdownLoader(
#             path,
#             glob="./*.md",
#             loader_cls=TextLoader,
#             show_progress=True,
#             use_multithreading=True,
#         )


#         doc = loader.load()
#         documents.append(doc)



# loader = DirectoryLoader(
#     '.',
#     glob="*.pdf",
#     loader_cls=PyPDFLoader,
#     show_progress=True,
#     use_multithreading=True,
# )

# documents = loader.load()

100%|██████████| 386/386 [00:00<00:00, 3146.87it/s]


In [10]:
print(f'We have {len(documents)} pages in total')

We have 386 pages in total


In [ ]:
# for doc in documents:

#         print(doc.page_content)

In [ ]:
# for page in range(len(documents)):
#     page_content = documents[page].page_content

#     # if 'ак работают платеж' in page_content:
#     #     print(page_content)
#     page_content = page_content.replace('help/assets/images/', 'https://www.rustore.ru/help/assets/images/')
#     documents[page].page_content = page_content

In [12]:
headers_to_split_on = [
    ("#", "Header 1"),
    # ("##", "Header 2"),
    # ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,strip_headers=True)

all_splits = []

for doc in documents:
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    all_splits.extend(md_header_splits)
len(all_splits)

388

In [13]:
# one_big_split = all_splits[0].copy()
# for split in all_splits[1:]:
#     one_big_split.page_content += '\n' + split.page_content

In [14]:
# one_big_split

In [15]:
# all_splits = [split for split in all_splits if len(split.page_content) > 70]

In [16]:
# len(, len(texts)

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

texts = text_splitter.split_documents(all_splits)
# texts = [text for text in texts if len(text.page_content) > 200]
# texts = text_splitter.split_documents([one_big_split])
texts = [text for text in texts if len(text.page_content) > 200]

for i, text in enumerate(texts):
    texts[i].page_content = ' '.join([m for k, m in text.metadata.items()])+ ' ' + texts[i].page_content

In [18]:
len(texts)

3023

In [ ]:
# for doc in documents:
#     # if 'тистика загрузки верс' in doc.page_content:
#     # if 'просмотреть статистику загрузки версий?' in doc.page_content:
#     if 'ак обновить приложения в RuSt' in doc.page_content:
#         print(doc.page_content)

In [ ]:
# for text in texts:
#     if 'просмотреть статистику загрузки ' in text.page_content:
#         print(text.page_content)

In [ ]:
# import pickle
# with open('md_splits.pickle', 'wb') as f:
#     pickle.dump(all_splits, f)

# Embeddings

In [19]:
class CFG:
    # LLMs
    model_name = 'distilbert/distilgpt2' # wizardlm, llama2-7b-chat, llama2-13b-chat, mistral-7B
    temperature = 1e-4 # 0
    top_p = 0.9
    repetition_penalty = 1.15

    # splitting
    # split_chunk_size = 1200
    # split_overlap = 200

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2' # 'cointegrated/LaBSE-en-ru' # # #  # 'DeepPavlov/rubert-base-cased-sentence' #  #'cointegrated/LaBSE-en-ru' #
    # embeddings_model_repo =  'intfloat/multilingual-e5-large-instruct' # # 'DeepPavlov/rubert-base-cased-sentence' # 'Den4ikAI/sbert_large_mt_ru_retriever' #'sentence-transformers/all-MiniLM-L6-v2' # 'all-mpnet-base-v2' # 'all-mpnet-base-v2' # 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 4

    # paths
    PDFs_path = 'docs/'
    # PDFs_path = '/content'
    Embeddings_path =  'faiss-hp'
    Output_folder = 'doc_vectordb'

In [20]:
%%time


embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
    # model_kwargs = {"device": "cpu"}
)

# %pip install yandexcloud
# import os
# os.environ["YC_API_KEY"] = "AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"

# from langchain_community.embeddings.yandex import YandexGPTEmbeddings
# embeddings = YandexGPTEmbeddings(YC_API_KEY = 'AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr', folder_id='b1gvmo70ll74cvokevfk', model_kwargs = {"device": "cuda"})



# %pip install -q yandex_chain

# from yandex_chain import YandexLLM, YandexEmbeddings

# embeddings = YandexEmbeddings(api_key="AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr", folder_id='b1g72uajlds114mlufqi')

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 1.28 s, sys: 716 ms, total: 1.99 s
Wall time: 10.3 s


In [21]:
# truncate =

In [22]:
# %%time
# FAISS.from_documents(
#     documents = texts[:20], # [:truncate]
#     embedding = embeddings
# )

In [23]:
# len(texts) / 20 * 90 / 3600

In [24]:
# for text in texts:
#     if 'Вы можете создать' in text.page_content:
#         print(text)

In [25]:
### create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts, # [:truncate]
    embedding = embeddings
)

### persist vector database
# vectordb.save_local(f"{CFG.Output_folder}/faiss_index_hp") # save in output folder
vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_hp") # save in input folder

In [ ]:
# !zip -r embeddings_full_doc.zip /content/faiss-hp

In [ ]:
# [a.page_content for a in texts[:truncate]]

# Retriever

In [26]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [27]:
%%time

### download embeddings model
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name = CFG.embeddings_model_repo,
#     model_kwargs = {"device": "cuda"}
# )

# # load vector DB embeddings
# vectordb = FAISS.load_local(
#     CFG.Embeddings_path + '/faiss_index_hp', # from input folder
# #     CFG.Output_folder + '/faiss_index_hp', # from output folder
#     embeddings,
#     allow_dangerous_deserialization=True
# )

clear_output()

CPU times: user 1.55 ms, sys: 47 µs, total: 1.6 ms
Wall time: 1.66 ms


In [28]:
res = vectordb.similarity_search('Как просмотреть статистику загрузки версий?')
res

[Document(metadata={'Header 1': 'Сценарии использования'}, page_content='Сценарии использования В этом разделе собраны основные сценарии работы с приложениями в RuStore.  \n### Вы узнате  \n* [Как работать с платежами](/help/guides/payments)\n* [Как настроить подписание приложения](/help/guides/sign-apk)'),
 Document(metadata={'Header 1': 'История обновлений'}, page_content='История обновлений ## Версия SDK 2.0.0  \n* Версия SDK review 2.+.  \n## Версия SDK 1.0.1  \n* Внутреннее обновление SDK.  \n## Версия SDK 1.0  \n* Добавлены асинхронные варианты Blueprint-методов вместо подписок на события.\n* Багфиксы.'),
 Document(metadata={'Header 1': 'Общий процесс работы с отзывами'}, page_content='Общий процесс работы с отзывами Возможные варианты кодов ошибок  \n| Код | Значение |\n| --- | --- |\n| `200` | OK |\n| `400` | Неправильный запрос |\n| `401` | Не авторизован |\n| `404` | Не найдено |\n| `500` | Внутренняя ошибка сервера |  \nПример ответа в случае ошибки  \n```\n{\n"code": "ERROR

In [29]:
!pip install rank_bm25

In [30]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import Chroma
from langchain_community.retrievers import TFIDFRetriever
from langchain_community.retrievers.bm25 import BM25Retriever

# retriever_base =  TFIDFRetriever.from_documents(texts, k=CFG.k)
retriever_base =  BM25Retriever.from_documents(texts, k=CFG.k)
retriever_advanced = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_base, retriever_advanced], weights=[0.5, 0.5],k = CFG.k)

In [31]:
# question = "Как создать аккаунт разработчика?"
# question = 'Как просмотреть статистику загрузки версий?'
question = "Ошибка что значит RuStoreReviewExists?"
# question = "Регистрация разработчиков в RuStore: шаги и опции "
# question = "Как добавить дополнительного разработчика?"
# question = "Как подписаться на топик?"
# question = 'Создать новую учётную запись для разработчика, а затем добавить его в компанию доступным в системе людям'
# question = 'как интегрировать SDK Remote Config для Kotlin'
# question = 'собрать HAR-логи​'
# question = 'Пример отчета по монетизации'
# question = 'Как подключить монетизацию?'
# question = 'Где находится политика конфиденциальности компании?'
# question = 'как использовать VK ID сдк'
similar = ensemble_retriever.invoke(question, k = CFG.k)
# similar = retriever_base.invoke(question, k = CFG.k)
similar

[Document(metadata={'Header 1': 'Вопросы и ответы'}, page_content='Вопросы и ответы к сведениюНастоящий раздел содержит примеры кода для Kotlin.  \n**Q: Как исправить ошибку «Application is not verified yet»?**  \nА: Ошибка возникает в следующих случаях:  \n* приложение не прошло модерацию в [Консоли RuStore](https://console.rustore.ru/sign-in);\n* тестируемая разработчиком APK не совпадает с APK, загруженной в [Консоль RuStore](https://console.rustore.ru/sign-in).  \nВторой пункт нужно перепроверить следующим образом.  \n* `applicationId`, указанный в `build.gradle`, должен совпадать с `applicationId` APK-файла, который вы публиковали в [Консоли RuStore](https://console.rustore.ru/sign-in).\n* подпись `keystore` должна совпадать с подписью, которой было подписано приложение, опубликованное в [Консоли RuStore](https://console.rustore.ru/sign-in). Убедитесь, что используемый `buildType` (например, `debug`) использует такую же подпись, что и опубликованное приложение (например, `release`

## Evaluate

In [32]:
import pandas as pd
questions = pd.read_json('questions_for_model.json')

for question in questions.index:
    similar = ensemble_retriever.invoke(question, k = 1)[0]
    print(question, '\n', similar.page_content, '\n' + '-'*50)

Как создать аккаунт разработчика? 
 RuStore: Авторизация через VK ID - Подробная инструкция Авторизуйтесь в RuStore, чтобы из него устанавливать, обновлять или открывать приложения. Вы можете авторизоваться в RuStore с аккаунтом [VK ID](https://connect.vk.com/promo?).  
## Как войти с аккаунтом VK ID  
1. Нажмите ![icon](/help/rustore-icon.svg) на домашнем экране вашего устройства.
2. Нажмите «Войти».
3. Чтобы авторизоваться с VK ID из приложения ВКонтакте, нажмите «Продолжить».  
Если на вашем устройстве установлено приложение ВКонтакте и вы авторизованы в нем, то ваши данные автоматически отобразятся в RuStore. Выберите один из доступных аккаунтов и войдите в систему.  
![img](/help/assets/images/Hello+ SberID-c4b1b31ee9991bc87814dbe84eac30fd.webp)
![img](/help/assets/images/Android Screen-9609e9c959cac04881f2da8e1f44e631.webp)
## Как войти в RuStore без ВКонтакте  
В RuStore можно войти через [Сбер ID](/help/users/authorization/login-sber-id), [Яндекс ID](/help/users/authorization/l

In [75]:
import pandas as pd
questions = pd.read_json('questions_for_model.json')

for question in questions.index:
    similar = retriever_advanced.invoke(question, k = 1)[0]
    print(question, '\n', similar.page_content, '\n' + '-'*50)

Как создать аккаунт разработчика? 
 Сценарии использования В этом разделе собраны основные сценарии работы с приложениями в RuStore.  
### Вы узнате  
* [Как работать с платежами](/help/guides/payments)
* [Как настроить подписание приложения](/help/guides/sign-apk) 
--------------------------------------------------
Как узнать VK ID? 
 Сценарии использования В этом разделе собраны основные сценарии работы с приложениями в RuStore.  
### Вы узнате  
* [Как работать с платежами](/help/guides/payments)
* [Как настроить подписание приложения](/help/guides/sign-apk) 
--------------------------------------------------
Как опубликовать своё приложение? 
 Сценарии использования В этом разделе собраны основные сценарии работы с приложениями в RuStore.  
### Вы узнате  
* [Как работать с платежами](/help/guides/payments)
* [Как настроить подписание приложения](/help/guides/sign-apk) 
--------------------------------------------------
У нашего приложения сменился владелец, Как изменить владельца 

In [34]:
import pandas as pd
questions_orgs = pd.read_excel('Пример вопросов с ответами с испольхованием хэлпы.xlsx')

for question in questions_orgs['Вопрос']:
    similar = ensemble_retriever.invoke(question, k = 1)[0]
    print(question, '\n', similar.page_content, '\n' + '-'*50)

FileNotFoundError: [Errno 2] No such file or directory: 'Пример вопросов с ответами с испольхованием хэлпы.xlsx'

# Model

In [99]:
import requests

# query = """
# Чтобы ответить на вопрос, используй только следующие фрагменты контекста.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа нет в контексте, напиши 'уточните, пожалуйста, вопрос'.

# query = """
# Ты получишь контекст и вопрос по нему.
# Выведи ответ на вопрос, используя только контекст.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа нет в контексте, напиши 'уточните, пожалуйста, вопрос', не выдумывай его сам!!!
# """

# query = """
# Ты получишь контекст и вопрос по нему, который касается магазина приложений RuStore.
# Выведи ответ на вопрос, используя только контекст.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа нет в контексте, напиши 'уточните, пожалуйста, вопрос', не выдумывай его сам!!!
# На вопросы не по теме RuStore тоже отвечай так (например, вопрос 'как тебя зовут' не относятся к теме)
# """

# query = """
# Ты получишь контекст и вопрос по нему, который касается магазина приложений RuStore.
# Выведи ответ на вопрос, используя только контекст.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа нет в контексте, напиши 'уточните, пожалуйста, вопрос', не выдумывай его сам!!!
# На вопросы не по теме магазина приложений тоже отвечай 'уточните, пожалуйста, вопрос' (например, вопрос 'как тебя зовут' не относятся к теме)
# """

# query = """
# Ты получишь контекст и вопрос по нему, который касается магазина приложений RuStore.
# Выведи ответ на вопрос, используя только контекст.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа нет в контексте или вопрос не по теме магазина приложений, напиши 'уточните, пожалуйста, вопрос', не выдумывай ответ сам!!!
# Например, вопрос 'как тебя зовут?' не относится к теме, а вот 'как мне выпустить приложение?' - относится.
# """
# query = """Прочитайте предоставленный контекст (часть документации для разработчиков от магазина приложений RuStore).
# Прочитайте вопрос.
# Если ответ на вопрос содержится в контексте, напишите ответ на вопрос.
# Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
# Если ответа на вопрос нет в контексте или вопрос не относится к теме, напишите 'уточните, пожалуйста, вопрос'.
# """

query = """
Ты получишь контекст и вопрос по нему, который касается магазина приложений RuStore.
Выведи ответ на вопрос, используя только контекст.
Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.
Если ответа нет в контексте или вопрос не по теме магазина приложений, напиши 'уточните, пожалуйста, вопрос', не выдумывай ответ сам!!!
Например, вопрос 'как тебя зовут?' не относится к теме, а вот 'как мне выпустить приложение?' - относится.
"""



def yagpt_request(question, context):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"
    }

    ya_prompt = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": "1024"
        },
        "messages": [
            {
                "role": "system",
                "text": context
            },
            {
                "role": "user", "text":
    f"""Контекст:
    {context}

    Вопрос: {question}"""
            }
        ]
    }

    response = requests.post(url, headers=headers, json=ya_prompt)
    result = eval(response.text)

    # print(result)

    return result['result']['alternatives'][0]['message']['text']


def refactor_sim_search_res(question):
    retriever_ans_raw = ensemble_retriever.invoke(question, k=6)
    retriever_ans = '\n\n'.join([i.page_content for i in retriever_ans_raw])

    context = query.format(context=retriever_ans)

    text = yagpt_request(question, context)
    return text, retriever_ans_raw

In [100]:
questions_false = ['Че?',
             'Как дела?',
             'Где находится Россия?',
             'А как приготовить гречку?',
             'Займи денег',
             "Hello what's your name?",
             "Ты куда ушел???",
             "Почему ты не отвечаешь на мои вопросы???"]

In [101]:
for question in questions_false:
    ans = refactor_sim_search_res(question)
    print(question, '\n', ans[0])
    print('-'*70)

    # similar = ensemble_retriever.invoke(question, k = 1)[0]
    # print(question, '\n', similar.page_content, '\n' + '-'*50)

Че? 
 Уточните, пожалуйста, вопрос.
----------------------------------------------------------------------
Как дела? 
 Ответ:

Как дела?
----------------------------------------------------------------------
Где находится Россия? 
 Россия находится на северо-востоке Евразии.
----------------------------------------------------------------------
А как приготовить гречку? 
 Уточните, пожалуйста, вопрос.

Мы можем рассказать вам, как выпустить приложение в RuStore, но не как приготовить гречку.
----------------------------------------------------------------------
Займи денег 
 Уточните, пожалуйста, вопрос.
----------------------------------------------------------------------
Hello what's your name? 
 Уточните, пожалуйста, вопрос.
----------------------------------------------------------------------
Ты куда ушел??? 
 Уточните, пожалуйста, вопрос.
----------------------------------------------------------------------
Почему ты не отвечаешь на мои вопросы??? 
 Уточните, пожалуйста, вопрос

In [98]:
for question in questions.index:
    ans = refactor_sim_search_res(question)
    print(question, '\n', ans[0])
    print('-'*70)

Как создать аккаунт разработчика? 
 Чтобы создать аккаунт разработчика, необходимо выполнить следующие шаги:
1. Перейдите на официальный сайт RuStore.
2. Нажмите кнопку «Регистрация» в правом верхнем углу экрана.
3. Заполните форму регистрации, указав необходимые данные.
4. Подтвердите адрес электронной почты и номер телефона.
5. Получите SMS с кодом подтверждения.
6. Введите полученный код в соответствующее поле на сайте.
7. Создайте пароль для доступа к аккаунту.
8. После успешной регистрации вы получите электронное письмо со ссылкой для активации аккаунта.
9. Перейдите по ссылке в письме, чтобы активировать свой аккаунт.

После активации аккаунта вы сможете начать разработку приложений для RuStore и публиковать их в магазине.
----------------------------------------------------------------------
Как узнать VK ID? 
 **Как узнать VK ID?**

VK ID можно узнать, зайдя в свой профиль ВКонтакте. Идентификатор расположен в адресной строке браузера после «https://vk.com/».

Также вы можете п

In [ ]:
question = 'как интегрировать SDK Remote Config для Kotlin'

In [ ]:
ans, retriever_ans_raw = refactor_sim_search_res(question)
print(ans)

# Paraphrase

In [ ]:
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Ты получаешь на вход вопрос пользователя, который нужно переформулировать, чтобы по нему было легко найти нужную информацию в документации.
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Ты получаешь на вход вопрос пользователя, который нужно переформулировать, чтобы получится текст, с помощью которого можно будет быстро найти нужную информацию в документации по ключевым словам.
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# На вход получаешь вопрос пользователя, извлеки из него ключевую информацию, чтобы можно было по ключевым словам найти нужную информацию в докуменатации
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Перефразируй вопрос пользователя так, чтобы нужную информацию можно было легко найти по ключевым словам в документации.
# Пример: вопрос 'Можно ли и откуда доставать данные гендера и возраста человека, скачавшего приложение? Необходимо для более точной настройки таргета РСЯ' переформулируем в 'Данные гендера и возраста человека, скачавшего приложение'
# Напиши только переформулированный вопрос
# """
query_paraphrase = """
Ты помогаешь создать ИИ-помощника по документации RuSrore.
Перефразируй вопрос пользователя, удалив ненужную информацию
Пример: вопрос 'Можно ли и откуда доставать данные гендера и возраста человека, скачавшего приложение? Необходимо для более точной настройки таргета РСЯ' переформулируем в 'Получение данных гендера и возраста человека, скачавшего приложение'
Напиши только переформулированный вопрос
"""

def yagpt_rparaphrase(question):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"
    }

    ya_prompt = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": "1024"
        },
        "messages": [
            {
                "role": "system",
                "text": query_paraphrase
            },
            {
                "role": "user", "text": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=ya_prompt)
    result = eval(response.text)

    return result['result']['alternatives'][0]['message']['text']

In [ ]:
question = 'как интегрировать SDK Remote Config для Kotlin'
ans = yagpt_rparaphrase(question)
ans

## Evaluate paraphrase

In [ ]:
questions_orgs = pd.read_excel('/content/Пример вопросов с ответами с испольхованием хэлпы.xlsx')

In [ ]:
for question in questions_orgs['Вопрос']:
    question_paraphrased = yagpt_rparaphrase(question)
    similar = ensemble_retriever.invoke(question_paraphrased, k = 1)[0]
    print(question, '--->', question_paraphrased, '\n', similar.page_content, '\n' + '-'*50)

In [ ]:
for question in questions_orgs['Вопрос']:
    question_paraphrased = yagpt_rparaphrase(question)
    similar = ensemble_retriever.invoke(question_paraphrased, k = 1)[0]
    print(question, '--->', question_paraphrased, '\n', similar.page_content, '\n' + '-'*50)
    refactor_sim_search_res(question)

In [ ]:
for question in questions.index:
    question_paraphrased = yagpt_rparaphrase(question)
    retriever_ans = ensemble_retriever.invoke(question_paraphrased, k = 1)[:4]

    retriever_ans = '\n\n'.join([i.page_content for i in retriever_ans])
    context = query.format(context=retriever_ans)
    text = yagpt_request(question_paraphrased, context)


    print(question, '--->', question_paraphrased, '\n', text, '\n' + '-'*50)